In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Reviews.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
df.shape

(568454, 10)

In [4]:
df.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [5]:
df.duplicated().sum()

0

In [6]:
df = df.dropna()
df.shape

(568411, 10)

In [7]:
df.Score.value_counts()

5    363111
4     80655
1     52264
3     42638
2     29743
Name: Score, dtype: int64

In [8]:
df = df[df['Score'] != 3]
df.shape

(525773, 10)

In [9]:
df['Score'] = df['Score'].replace({1: 0, 2: 0, 4: 1, 5: 1})
df['Score'].value_counts()

1    443766
0     82007
Name: Score, dtype: int64

In [10]:
final = pd.DataFrame(df, columns = ['Text', 'Score'])
final.head()

,Text,Score
0,I have bought several of the Vitality canned d...,1
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,1
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,1


In [11]:
from bs4 import BeautifulSoup

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [13]:
ps = PorterStemmer()

In [14]:
from nltk.stem import WordNetLemmatizer

lem = WordNetLemmatizer()

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer()

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(final['Text'], final['Score'], test_size = 0.3, random_state = 0)

In [17]:
corpus_train = []
corpus_test  = []

for i in range(x_train.shape[0]):
    soup = BeautifulSoup(x_train.iloc[i], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_train.append(review)

In [18]:
for j in range(x_test.shape[0]):
    soup = BeautifulSoup(x_test.iloc[j], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_test.append(review)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(ngram_range=(1, 3))

tfidf_vec_train = tfidf_vec.fit_transform(corpus_train)
tfidf_vec_test = tfidf_vec.transform(corpus_test)

In [20]:
from sklearn.svm import LinearSVC
linear_svc = LinearSVC(C=0.5, random_state=42)

In [21]:
linear_svc.fit(tfidf_vec_train, y_train)

LinearSVC(C=0.5, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [22]:
predict = linear_svc.predict(tfidf_vec_test)

In [23]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [24]:
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.91      0.83      0.87     24567
           1       0.97      0.99      0.98    133165

    accuracy                           0.96    157732
   macro avg       0.94      0.91      0.92    157732
weighted avg       0.96      0.96      0.96    157732



In [25]:
print(confusion_matrix(y_test, predict))

[[ 20336   4231]
 [  1916 131249]]


In [26]:
print("Accuracy:", accuracy_score(y_test, predict))

Accuracy: 0.961028833717952
